In [39]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
num_epochs = 15
batch_size = 8
learning_rate = 0.01

# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [40]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [41]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x


In [42]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [43]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)


Epoch [1/15], Step [2000/6250], Loss: 1.8560
Epoch [1/15], Step [4000/6250], Loss: 1.8286
Epoch [1/15], Step [6000/6250], Loss: 1.4149
Epoch [2/15], Step [2000/6250], Loss: 2.4808
Epoch [2/15], Step [4000/6250], Loss: 1.4274
Epoch [2/15], Step [6000/6250], Loss: 1.0468
Epoch [3/15], Step [2000/6250], Loss: 1.3828
Epoch [3/15], Step [4000/6250], Loss: 1.2506
Epoch [3/15], Step [6000/6250], Loss: 1.7517
Epoch [4/15], Step [2000/6250], Loss: 0.8761
Epoch [4/15], Step [4000/6250], Loss: 0.5670
Epoch [4/15], Step [6000/6250], Loss: 1.4666
Epoch [5/15], Step [2000/6250], Loss: 0.6261
Epoch [5/15], Step [4000/6250], Loss: 1.0484
Epoch [5/15], Step [6000/6250], Loss: 1.1642
Epoch [6/15], Step [2000/6250], Loss: 1.1636
Epoch [6/15], Step [4000/6250], Loss: 1.1091
Epoch [6/15], Step [6000/6250], Loss: 1.1780
Epoch [7/15], Step [2000/6250], Loss: 1.2199
Epoch [7/15], Step [4000/6250], Loss: 1.1127
Epoch [7/15], Step [6000/6250], Loss: 0.9873
Epoch [8/15], Step [2000/6250], Loss: 1.3054
Epoch [8/1

In [44]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 63.53 %
Accuracy of plane: 63.4 %
Accuracy of car: 78.1 %
Accuracy of bird: 53.7 %
Accuracy of cat: 46.9 %
Accuracy of deer: 49.0 %
Accuracy of dog: 49.9 %
Accuracy of frog: 75.9 %
Accuracy of horse: 67.8 %
Accuracy of ship: 79.6 %
Accuracy of truck: 71.0 %
